In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import accuracy_score
from sklearn import linear_model

def forwardSelection(indep_X,dep_Y,n):
    # Initialize an empty set to hold selected features
    selected_features = set()
    # Iterate until desired number of features is selected
    for _ in range(n_features):
        # Initialize best score and feature
        best_score = 0
        best_feature = None
        # Iterate over remaining features
        for feature in X.columns:
            # Skip if feature is already selected
            if feature in selected_features:
                continue
            # Create a copy of selected features plus the current feature
            current_features = list(selected_features) + [feature]
            # Fit a logistic regression model using selected features
            X_selected = X[current_features]
            X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
            clf = LogisticRegression()
            clf.fit(X_train, y_train)
            # Calculate accuracy on test set
            accuracy = accuracy_score(y_test, clf.predict(X_test))
            # Update best score and feature if accuracy improves
            if accuracy > best_score:
                best_score = accuracy
                best_feature = feature
        # Add best feature to selected features
        selected_features.add(best_feature)
    return selected_features

In [2]:
def backwardSelection(indep_X, dep_Y, n):
    remaining_features = set(indep_X.columns)
    selected_features = set(indep_X.columns)
    for _ in range(n):
        best_score = 0
        worst_feature = None
        for feature in selected_features:
            current_features = list(remaining_features - {feature})
            X_selected = indep_X[current_features]
            X_train, X_test, y_train, y_test = train_test_split(X_selected, dep_Y, test_size=0.2, random_state=42)
            clf = LogisticRegression()
            clf.fit(X_train, y_train)
            accuracy = accuracy_score(y_test, clf.predict(X_test))
            if accuracy > best_score:
                best_score = accuracy
                worst_feature = feature
        selected_features.remove(worst_feature)
        remaining_features.remove(worst_feature)
    return selected_features
    
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    
def cm_prediction(classifier,X_test):
     y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
     from sklearn.metrics import confusion_matrix
     cm = confusion_matrix(y_test, y_pred)
        
     from sklearn.metrics import accuracy_score 
     from sklearn.metrics import classification_report 
        #from sklearn.metrics import confusion_matrix
        #cm = confusion_matrix(y_test, y_pred)
        
     Accuracy=accuracy_score(y_test, y_pred )
        
     report=classification_report(y_test, y_pred)
     return  classifier,Accuracy,report,X_test,y_test,cm

def logistic(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      
    
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'linear', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
   
def Navie(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm         
    
    
def knn(X_train,y_train,X_test):
           
        # Fitting K-NN to the Training set
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      


def random(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def featureSel_Classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf): 
    
    dataframe=pd.DataFrame(index=['ChiSquare'],columns=['Logistic','SVMl','SVMnl','KNN','Navie','Decision','Random'])
    for number,idex in enumerate(dataframe.index):      
        dataframe['Logistic'][idex]=acclog[number]       
        dataframe['SVMl'][idex]=accsvml[number]
        dataframe['SVMnl'][idex]=accsvmnl[number]
        dataframe['KNN'][idex]=accknn[number]
        dataframe['Navie'][idex]=accnav[number]
        dataframe['Decision'][idex]=accdes[number]
        dataframe['Random'][idex]=accrf[number]
    return dataframe

In [7]:
dataset1=pd.read_csv("prep_CKD.csv",index_col=None)

df2=dataset1

df2 = pd.get_dummies(df2, drop_first=True)
# Assuming df2 is your DataFrame containing the data
indep_X=df2.drop('classification_yes', axis=1)
dep_Y=df2['classification_yes']

selected_features = backwardSelection(indep_X, dep_Y, 1)
print("Selected Features:", selected_features)  

acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]

X_train, X_test, y_train, y_test = split_scalar(indep_X[list(selected_features)], dep_Y)
    
        
classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
acclog.append(Accuracy)

classifier,Accuracy,report,X_test,y_test,cm=svm_linear(X_train,y_train,X_test)  
accsvml.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=svm_NL(X_train,y_train,X_test)  
accsvmnl.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=knn(X_train,y_train,X_test)  
accknn.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Navie(X_train,y_train,X_test)  
accnav.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Decision(X_train,y_train,X_test)  
accdes.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
accrf.append(Accuracy)
    
result=featureSel_Classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf)

result

C:\Anaconda\envs\ai\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\Anaconda\envs\ai\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  

C:\Anaconda\envs\ai\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\Anaconda\envs\ai\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  

Selected Features: {'pot', 'sg_b', 'ba_present', 'sg_e', 'htn_yes', 'rbc_normal', 'sc', 'bp', 'pc_normal', 'dm_yes', 'bu', 'pe_yes', 'cad_yes', 'sod', 'age', 'sg_c', 'ane_yes', 'rc', 'pcc_present', 'pcv', 'sg_d', 'appet_yes', 'su', 'bgr', 'hrmo', 'al'}


,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.99,0.99,1.0,0.97,0.98,0.9,1.0


#6 result
          Logistic	SVMl	SVMnl	KNN	   Navie	Decision	Random
ChiSquare	1.0	    0.98	1.0	    0.97	0.94	   0.91	     1.0

#5 result
          Logistic	SVMl	SVMnl	KNN  	Navie	Decision	Random
ChiSquare	1.0	    0.98	1.0	    0.99	 0.97	 0.91	    0.99

#4 result
          Logistic	SVMl	SVMnl	KNN	   Navie	Decision	Random
ChiSquare	1.0	    0.99	0.98	0.95	0.98	0.9	     0.99

#3 result
	        Logistic	SVMl	SVMnl	KNN	   Navie	Decision	Random
ChiSquare	1.0	          1.0	1.0	     0.97	0.98	0.9	          1.0

#2 result
          Logistic	SVMl	SVMnl	KNN	   Navie	Decision	Random
ChiSquare	1.0	    1.0	     1.0	0.97	0.98	  0.9      	0.99